In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import glob
import mysql.connector
from mysql.connector import Error

## FvB

Audience needed:
+ all the people that purchased business shoes ever

In [2]:
%run ./cohorts_pipeline_fvb_v3.py

In [10]:
df_product = pd.read_csv('./FvB Weekly Performance Dashboard/Data/Analytics_20190114-20210120_COMPLETE.csv', delimiter=',', decimal='.')

In [12]:
df_product.head()

,Product Category (Enhanced E-commerce),Transaction ID,Product Revenue,Unique Purchases,Quantity,Avg. Price,Avg. QTY,Product Refund Amount,Basket-to-Detail Rate,Buy-to-Detail Rate
0,Herren/Schuhe/Schnürer,DE01361598,3959.20,16,16,247.45,1,0,0.00%,0.00%
1,Herren/Schuhe/Schnürer,DE01337076,2999.25,15,15,199.95,1,0,0.00%,0.00%
2,Herren/Schuhe/Schnürer,DE01349187,1959.60,8,8,244.95,1,0,0.00%,0.00%
3,Herren/Schuhe/Schnürer,DE01343086,1919.60,8,8,239.95,1,0,0.00%,0.00%
4,Herren/Schuhe/Schnürer,DE01368121,1919.60,8,8,239.95,1,0,0.00%,0.00%


In [11]:
df_backend = pd.read_csv('./FvB Weekly Performance Dashboard/Data/Orders_Backend.csv', delimiter=';', decimal='.')

In [13]:
df_backend.head()

,Ordernummer,Orderdatum,Klant voornaam,Klant achternaam,E-mail,Aflevermethode,Totaalprijs,Kanaal,Locale,Unnamed: 9,Status
0,DE01367852,2021-01-11 08:44:54,Alexander,Meiners,alexander_meiners@web.de,Thuisbezorgd,313.92,Floris van Bommel webshop Duitsland,de_DE,Nog te alloceren,Nog te alloceren
1,DE01367851,2021-01-11 08:29:07,Olaf,Pfeifer,pfeifer@gigaron.de,Thuisbezorgd,269.85,Floris van Bommel webshop Duitsland,de_DE,Levering bezig,Levering bezig
2,DE01367850,2021-01-11 07:51:48,Mark,van Ommeren,markvanommeren@icloud.com,Thuisbezorgd,269.95,Floris van Bommel webshop Duitsland,de_DE,Levering bezig,Levering bezig
3,DE01367748,2021-01-11 04:50:29,Hubert,Kinzler,hkinzler@web.de,Thuisbezorgd,167.96,Floris van Bommel webshop Duitsland,de_DE,Levering bezig,Levering bezig
4,DE01367659,2021-01-11 00:46:23,Borche,Siljanoski,Borce.siljanoski@gmail.com,Thuisbezorgd,139.96,Floris van Bommel webshop Duitsland,de_DE,Levering bezig,Levering bezig


In [14]:
df_backend = df_cleaning_source(df_backend)

In [15]:
df_backend.head()

,Merchant_Reference,Creation_Date,First_Name,Last_Name,Email,Revenue,Market
0,DE01367852,2021-01-11 08:44:54,Alexander,Meiners,alexander_meiners@web.de,313.92,de_DE
1,DE01367851,2021-01-11 08:29:07,Olaf,Pfeifer,pfeifer@gigaron.de,269.85,de_DE
2,DE01367850,2021-01-11 07:51:48,Mark,van Ommeren,markvanommeren@icloud.com,269.95,de_DE
3,DE01367748,2021-01-11 04:50:29,Hubert,Kinzler,hkinzler@web.de,167.96,de_DE
4,DE01367659,2021-01-11 00:46:23,Borche,Siljanoski,Borce.siljanoski@gmail.com,139.96,de_DE


In [16]:
df_backend = first_order(df_backend)

In [17]:
df_backend.head()

,Email,Merchant_Reference,Creation_Date,First_Name,Last_Name,Revenue,Market,Creation_Date_YM,First_Order,First_Order_YM,Customer_Type
0,alexander_meiners@web.de,DE01367852,2021-01-11 08:44:54,Alexander,Meiners,313.92,de_DE,2021-01,2017-01-10 15:46:44,2017-01,Returning
1,pfeifer@gigaron.de,DE01367851,2021-01-11 08:29:07,Olaf,Pfeifer,269.85,de_DE,2021-01,2016-05-30 17:01:13,2016-05,Returning
2,markvanommeren@icloud.com,DE01367850,2021-01-11 07:51:48,Mark,van Ommeren,269.95,de_DE,2021-01,2021-01-11 07:51:48,2021-01,New
3,hkinzler@web.de,DE01367748,2021-01-11 04:50:29,Hubert,Kinzler,167.96,de_DE,2021-01,2021-01-03 12:14:08,2021-01,Returning
4,borce.siljanoski@gmail.com,DE01367659,2021-01-11 00:46:23,Borche,Siljanoski,139.96,de_DE,2021-01,2020-08-26 11:37:59,2020-08,Returning


In [19]:
df_backend.rename(columns={'Merchant_Reference':'Transaction ID'}, inplace=True)
df_backend

,Email,Transaction ID,Creation_Date,First_Name,Last_Name,Revenue,Market,Creation_Date_YM,First_Order,First_Order_YM,Customer_Type
0,alexander_meiners@web.de,DE01367852,2021-01-11 08:44:54,Alexander,Meiners,313.92,de_DE,2021-01,2017-01-10 15:46:44,2017-01,Returning
1,pfeifer@gigaron.de,DE01367851,2021-01-11 08:29:07,Olaf,Pfeifer,269.85,de_DE,2021-01,2016-05-30 17:01:13,2016-05,Returning
2,markvanommeren@icloud.com,DE01367850,2021-01-11 07:51:48,Mark,van Ommeren,269.95,de_DE,2021-01,2021-01-11 07:51:48,2021-01,New
3,hkinzler@web.de,DE01367748,2021-01-11 04:50:29,Hubert,Kinzler,167.96,de_DE,2021-01,2021-01-03 12:14:08,2021-01,Returning
4,borce.siljanoski@gmail.com,DE01367659,2021-01-11 00:46:23,Borche,Siljanoski,139.96,de_DE,2021-01,2020-08-26 11:37:59,2020-08,Returning
...,...,...,...,...,...,...,...,...,...,...,...
92673,merkel74@yahoo.de,DE01000302,2016-01-25 22:01:34,rainer,merkel,199.90,de_DE,2016-01,2016-01-25 22:01:34,2016-01,New
92674,rw@aixact.net,DE01000401,2016-01-25 21:43:03,Ralf,Wetzler,269.90,de_DE,2016-01,2016-01-25 21:43:03,2016-01,New
92675,service@jul.berlin,DE01000400,2016-01-25 21:24:48,JÖRG-UWE,LEHMANN,609.70,de_DE,2016-01,2016-01-25 21:24:48,2016-01,New
92676,maartenbongaerts@vanbommel.nl,DE01000200,2016-01-21 13:51:20,Ruurd,van der Weide,319.80,de_DE,2016-01,2016-01-20 16:30:07,2016-01,Returning


### Joining tables to get audience

In [32]:
temp1 = df_product.merge(df_backend, on='Transaction ID', how='left')
temp1.head()

,Product Category (Enhanced E-commerce),Transaction ID,Product Revenue,Unique Purchases,Quantity,Avg. Price,Avg. QTY,Product Refund Amount,Basket-to-Detail Rate,Buy-to-Detail Rate,Email,Creation_Date,First_Name,Last_Name,Revenue,Market,Creation_Date_YM,First_Order,First_Order_YM,Customer_Type
0,Herren/Schuhe/Schnürer,DE01361598,3959.20,16,16,247.45,1,0,0.00%,0.00%,diehl.rico@googlemail.com,2020-11-30 21:09:15,Rico,Diehl,1159.77,de_DE,2020-11,2017-11-10 17:47:41,2017-11,Returning
1,Herren/Schuhe/Schnürer,DE01337076,2999.25,15,15,199.95,1,0,0.00%,0.00%,m.meinhart@t-online.de,2020-05-01 12:29:21,Mirko,Meinhart,463.83,de_DE,2020-05,2017-11-14 19:12:48,2017-11,Returning
2,Herren/Schuhe/Schnürer,DE01349187,1959.60,8,8,244.95,1,0,0.00%,0.00%,kai.baenfer@gmail.com,2020-08-18 09:26:07,Kai,Bänfer,895.82,de_DE,2020-08,2020-08-18 09:26:07,2020-08,New
3,Herren/Schuhe/Schnürer,DE01343086,1919.60,8,8,239.95,1,0,0.00%,0.00%,info@rottmann-ostendorf.lvm.de,2020-06-24 10:03:54,Ludger,Rottmann,191.96,de_DE,2020-06,2020-06-13 12:04:11,2020-06,Returning
4,Herren/Schuhe/Schnürer,DE01368121,1919.60,8,8,239.95,1,0,0.00%,0.00%,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [33]:
temp1.shape

(6378, 20)

In [ ]:
# Audience Business Shoes buyers

In [34]:
audience_1 = temp1.loc[:,['Email']]
audience = audience_1.drop_duplicates().reset_index(drop=True)

In [35]:
audience.shape

(5073, 1)

In [39]:
audience.head()

,Email
0,diehl.rico@googlemail.com
1,m.meinhart@t-online.de
2,kai.baenfer@gmail.com
3,info@rottmann-ostendorf.lvm.de
4,NaN


In [ ]:
# Audience Gasconne

In [59]:
audience_1 = unstack_table[(~unstack_table.email.isin(over12m.email.unique())) & (~unstack_table.email.isin(unsubs.email.unique()))].loc[:,['email']]

In [60]:
audience_1.shape

(52720, 1)

In [62]:
audience = pd.concat([audience_1, non_buyers]).drop_duplicates().reset_index(drop=True)

In [63]:
audience.shape

(72547, 1)

In [40]:
audience.dropna(inplace=True)
audience.head()

,Email
0,diehl.rico@googlemail.com
1,m.meinhart@t-online.de
2,kai.baenfer@gmail.com
3,info@rottmann-ostendorf.lvm.de
5,behrens_marc@gmx.de


In [41]:
audience.shape

(5072, 1)

In [43]:
audience.to_csv('./Outputs/FvB_Audience_Business_Shoes_21012021.csv', index=False)

## Dumping Data

In [31]:
#from gspread_pandas import Spread, Client

In [32]:
#fvb_mt_sheet = Spread('Data Dump - PC + FvB MT')

In [33]:
# Push Transactions
#fvb_mt_sheet.df_to_sheet(trans_month_de, index=True, sheet='FvB', start='A1', replace=True)